In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [83]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

ibmqfactory.load_account:WARNING:2022-12-08 00:30:49,167: Credentials are already in use. The existing account in the session will be replaced.


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [84]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [85]:
simulator = provider.get_backend('ibmq_qasm_simulator')

In [86]:
device.configuration().to_dict()['max_shots']

100000

In [87]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [88]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [89]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [90]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [91]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [92]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_20299/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [93]:
orign_circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘           │        »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«     ┌┴────────────┴┐   ┌────────────┐                  
«q_1: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘  ┌┴────────────┴┐ ┌──────────────┐
«q_2: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_3: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«                     └─────────────────┘ └────────────┘ 
«q_4: ───────────────────────────────────────────────────
«

In [94]:
#checking circuit

In [95]:
pauli_check_circs = []
phase_y_front = [0.008239051732521298, 0.013635804359443573, 0.05162548888524782, -2.1999413989625588e-05]
phase_z_front = [-0.010143905866716232, -0.028142425222093263, 0.008659821134257232, 1.4378429162081177]
phase_y_end = [0.008261085257040952, 0.013555948647231175, 0.0516520269898158, -4.555585169085397e-05]
phase_z_end = [1.3183908530236275, 1.3352666291100785, -0.26460520551963923, 1.2198754282688027]
x_front = [True, True, False, False]
for i in range(0, 4):
    circuit = QuantumCircuit(5)
    apply_checking_circuit(circuit, [i], [4], side = 'front', phase_y = phase_y_front[i], phase_z = phase_z_front[i], x = x_front[i])
    circuit += orign_circuit
    apply_checking_circuit(circuit, [i], [4], side = 'end', phase_y = phase_y_end[i], phase_z = phase_z_end[i])
    pauli_check_circs.append(circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_20299/1329398966.py:10: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [96]:
pauli_check_circs[-1].draw()

┌───┐       ┌───────────────┐┌───────────────┐               »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────■───────»
            ├───┤       └┬──────────────┤├───────────────┤       │       »
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├───────■───────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘               »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                 ┌─────────────┐»
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─────────■────────┤ Rz(-1.4378) ├»
     └──────┬───┬──────┘  └────────────┘         │        └─────────────┘»
q_4: ───────┤ H ├────────────────────────────────■───────────────────────»
            └───┘                                                        »
«     ┌───────────────┐    ┌────────────┐                                      »
«q_0: ┤ Ry(0.0082611) ├────┤ Rz(1.3184) ├──────────────────────────────────────»
«     └───────────────┘   ┌┴────────────┴┐ ┌────────────┐                      »
«q_1: ────────■───────────┤ Ry(0.013556) ├─┤ Rz(1.3353) ├──────────────────────»
«             │           └──────────────┘ └────────────┘     ┌──────────────┐ »
«q_2: ────────■───────────────────────────────────────────■───┤ Ry(0.051652) ├─»
«     ┌────────────────┐┌─────────────────┐┌────────────┐ │ ┌─┴──────────────┴┐»
«q_3: ┤ Ry(2.1999e-05) ├┤ Ry(-2.1999e-05) ├┤ Rz(1.4378) ├─■─┤ Ry(-4.5556e-05) ├»
«     └────────────────┘└─────────────────┘└────────────┘   └─────────────────┘»
«q_4: ─────────────────────────────────────────────────────────────────────────»
«                                                                              »
«                                                                            »
«q_0: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«q_1: ───────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐                                                       »
«q_2: ┤ Rz(-0.26461) ├───────────────────────────────────────────────────────»
«     └┬────────────┬┘┌─────────────┐┌────────────────┐   ┌─────────────────┐»
«q_3: ─┤ Rz(1.2199) ├─┤ Rz(-1.2199) ├┤ Ry(4.5556e-05) ├─■─┤ Ry(-4.5556e-05) ├»
«      └────────────┘ └─────────────┘└────────────────┘ │ └──────┬───┬──────┘»
«q_4: ──────────────────────────────────────────────────■────────┤ H ├───────»
«                                                                └───┘       »
«                   
«q_0: ──────────────
«                   
«q_1: ──────────────
«                   
«q_2: ──────────────
«     ┌────────────┐
«q_3: ┤ Rz(1.2199) ├
«     └────────────┘
«q_4: ──────────────
«

In [97]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [98]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4, num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
q_4: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXX
     ┌─

In [99]:
pauli_check_circs[1].draw()

┌───┐       ┌───────────────┐┌───────────────┐                   »
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├───────────────────»
            ├───┤       └┬──────────────┤├───────────────┤   ┌──────────────┐»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─┤ Rz(0.028142) ├»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ └──────────────┘»
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«                                                                    »
«q_0: ─────────────────────────────────────────────────────────────■─»
«     ┌───────────────┐┌───┐┌───┐┌──────────────┐┌───────────────┐ │ »
«q_1: ┤ Ry(-0.013636) ├┤ X ├┤ X ├┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─»
«     └───────────────┘└───┘└───┘└──────────────┘└───────────────┘   »
«q_2: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_3: ───────────────────────────────────────────────────────────────»
«                                                                    »
«q_4: ───────────────────────────────────────────────────────────────»
«                                                                    »
«     ┌───────────────┐ ┌────────────┐                                    »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├────────────────────────────────────»
«     └───────────────┘┌┴────────────┴┐   ┌────────────┐  ┌─────────────┐ »
«q_1: ────────■────────┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──┤ Rz(-1.3353) ├─»
«             │        └──────────────┘  ┌┴────────────┴┐ ├─────────────┴┐»
«q_2: ────────■───────────────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├»
«                             │        ┌─┴──────────────┴┐└┬────────────┬┘»
«q_3: ────────────────────────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─»
«                                      └─────────────────┘ └────────────┘ »
«q_4: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                                                       
«q_0: ──────────────────────────────────────────────────
«     ┌───────────────┐   ┌──────────────┐┌────────────┐
«q_1: ┤ Ry(-0.013556) ├─■─┤ Ry(0.013556) ├┤ Rz(1.3353) ├
«     └───────────────┘ │ └──────────────┘└────────────┘
«q_2: ──────────────────┼───────────────────────────────
«                       │                               
«q_3: ──────────────────┼───────────────────────────────
«                       │      ┌───┐                    
«q_4: ──────────────────■──────┤ H ├────────────────────
«                              └───┘

In [100]:
circuit_list = []
for check_circ in pauli_check_circs:
    for meas in meas_qcs:
        temp_circuit = check_circ.copy()
        temp_circuit += meas
        temp_circuit.measure_active()
        circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_20299/1839173154.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [101]:
len(circuit_list)

100

In [102]:
circuit_list[2].draw()

┌───┐       ┌───────────────┐┌───────────────┐   »
      q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─»
                  ├───┤       └┬──────────────┤├───────────────┤ │ »
      q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─»
             ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │ »
      q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─»
           ┌─┴──────────────┴┐└─┬────────────┬┘                  │ »
      q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─»
           └──────┬───┬──────┘  └────────────┘                   │ »
      q_4: ───────┤ H ├──────────────────────────────────────────■─»
                  └───┘                                            »
measure_0: ════════════════════════════════════════════════════════»
                                                                   »
measure_1: ════════════════════════════════════════════════════════»
                                                                   »
measure_2: ════════════════════════════════════════════════════════»
                                                                   »
measure_3: ════════════════════════════════════════════════════════»
                                                                   »
measure_4: ════════════════════════════════════════════════════════»
                                                                   »
«           ┌──────────────┐┌────────────────┐┌───┐┌───┐┌───────────────┐»
«      q_0: ┤ Rz(0.010144) ├┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├»
«           └──────────────┘└────────────────┘└───┘└───┘└───────────────┘»
«      q_1: ─────────────────────────────────────────────────────────────»
«                                                                        »
«      q_2: ─────────────────────────────────────────────────────────────»
«                                                                        »
«      q_3: ─────────────────────────────────────────────────────────────»
«                                                                        »
«      q_4: ─────────────────────────────────────────────────────────────»
«                                                                        »
«measure_0: ═════════════════════════════════════════════════════════════»
«                                                                        »
«measure_1: ═════════════════════════════════════════════════════════════»
«                                                                        »
«measure_2: ═════════════════════════════════════════════════════════════»
«                                                                        »
«measure_3: ═════════════════════════════════════════════════════════════»
«                                                                        »
«measure_4: ═════════════════════════════════════════════════════════════»
«                                                                        »
«           ┌───────────────┐   ┌───────────────┐ ┌────────────┐ »
«      q_0: ┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├─»
«           └───────────────┘ │ └───────────────┘┌┴────────────┴┐»
«      q_1: ──────────────────■─────────■────────┤ Ry(0.013556) ├»
«                                       │        └──────────────┘»
«      q_2: ────────────────────────────■───────────────■────────»
«                                                       │        »
«      q_3: ────────────────────────────────────────────■────────»
«                                                                »
«      q_4: ─────────────────────────────────────────────────────»
«                                                                »
«measure_0: ═════════════════════════════════════════════════════»
«                                                                »
«measure_1: ═════════════════════════════════════════════════════»
«                                                       

In [103]:
orign_qc = QuantumCircuit.from_qasm_file("qasm/vqe_LiH.qasm")

In [104]:
#generate individual measurment circuits that changes the basis
meas_orign_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4, num_qargs = 4)
    meas_orign_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
YYXX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_3: ─┤ U(π/2,0,π) ├─
      └

In [105]:
orign_circuit_list = []
for meas in meas_orign_qcs:
    temp_circuit = orign_qc.copy()
    temp_circuit += meas
    temp_circuit.measure_active()
    orign_circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_20299/242718267.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [106]:
len(orign_circuit_list)

25

In [107]:
circuit_list = circuit_list + orign_circuit_list

In [108]:
len(circuit_list)

125

In [109]:
# max_shots = device.configuration().to_dict()['max_shots']
# max_shots

In [110]:
max_shots = 10000

In [111]:
trans_qcs = transpile(circuit_list, device, optimization_level = 3, seed_transpiler = 0)

In [112]:
trans_qcs[0].draw()

»
  ancilla_0 -> 0 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_1 -> 1 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_2 -> 2 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_3 -> 3 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_4 -> 4 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_5 -> 5 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_6 -> 6 ─────────────────────────────────────────────────────»
                                                                      »
  ancilla_7 -> 7 ─────────────────────────────────────────────────────»
                   ┌─────────┐      ┌────┐                            »
        q_4 -> 8 ──┤ Rz(π/2) ├──────┤ √X ├────────────────────────────»
                   └─────────┘      └────┘                            »
  ancilla_8 -> 9 ─────────────────────────────────────────────────────»
                                                                      »
 ancilla_9 -> 10 ─────────────────────────────────────────────────────»
                 ┌─────────────┐    ┌────┐   ┌─────────────┐  ┌────┐  »
       q_0 -> 11 ┤ Rz(-1.5695) ├────┤ √X ├───┤ Rz(-1.5708) ├──┤ √X ├──»
                 └┬────────────┤    ├────┤   └─┬──────────┬┘  ├────┤  »
       q_3 -> 12 ─┤ Rz(1.5695) ├────┤ √X ├─────┤ Rz(-π/2) ├───┤ √X ├──»
                  ├────────────┤    ├────┤   ┌─┴──────────┴┐  ├────┤  »
       q_2 -> 13 ─┤ Rz(1.5695) ├────┤ √X ├───┤ Rz(-1.5707) ├──┤ √X ├──»
                  └───┬────┬───┘┌───┴────┴──┐└────┬────┬───┘┌─┴────┴─┐»
       q_1 -> 14 ─────┤ √X ├────┤ Rz(3.128) ├─────┤ √X ├────┤ Rz(-π) ├»
                      └────┘    └───────────┘     └────┘    └────────┘»
ancilla_10 -> 15 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_11 -> 16 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_12 -> 17 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_13 -> 18 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_14 -> 19 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_15 -> 20 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_16 -> 21 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_17 -> 22 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_18 -> 23 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_19 -> 24 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_20 -> 25 ─────────────────────────────────────────────────────»
                                                                      »
ancilla_21 -> 26 ─────────────────────────────────────────────────────»
                                                                      »
      measure_0: ═════════════════════════════════════════════════════»
                                      

In [113]:
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_1 -> 1 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_2 -> 2 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_3 -> 3 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_4 -> 4 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_5 -> 5 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_6 -> 6 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_7 -> 7 ──────────────────────────────────────────────────────────────»
                      ┌────┐      ┌────────────┐      ┌────┐    ┌─────────────┐»
        q_3 -> 8 ─────┤ √X ├──────┤ Rz(3.1416) ├──────┤ √X ├────┤ Rz(-4.8453) ├»
                      └────┘      └────────────┘      └────┘    └─────────────┘»
  ancilla_8 -> 9 ──────────────────────────────────────────────────────────────»
                                                                               »
 ancilla_9 -> 10 ──────────────────────────────────────────────────────────────»
                 ┌─────────────┐      ┌────┐     ┌─────────────┐     ┌────┐    »
       q_2 -> 11 ┤ Rz(-3.1329) ├──────┤ √X ├─────┤ Rz(-1.6224) ├─────┤ √X ├────»
                 └─────────────┘      └────┘     └─────────────┘     └────┘    »
ancilla_10 -> 12 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_11 -> 13 ──────────────────────────────────────────────────────────────»
                      ┌────┐    ┌───────────────┐     ┌────┐                   »
       q_1 -> 14 ─────┤ √X ├────┤ Rz(-0.013636) ├─────┤ √X ├───────────────────»
                      └────┘    └───────────────┘     └────┘                   »
ancilla_12 -> 15 ──────────────────────────────────────────────────────────────»
                  ┌────────────┐      ┌────┐     ┌─────────────┐     ┌────┐    »
       q_0 -> 16 ─┤ Rz(1.5695) ├──────┤ √X ├─────┤ Rz(-1.5708) ├─────┤ √X ├────»
                  └────────────┘      └────┘     └─────────────┘     └────┘    »
ancilla_13 -> 17 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_14 -> 18 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_15 -> 19 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_16 -> 20 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_17 -> 21 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_18 -> 22 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_19 -> 23 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_20 -> 24 ──────────────────────────────────────────────────────────────»
                             

In [114]:
len(trans_qcs)

125

In [115]:
for qc in trans_qcs:
    print(qc.count_ops())

OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 29), ('sx', 22), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 29), ('sx', 22), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz', 30), ('sx', 22), ('cx', 5), ('measure', 5), ('x', 1), ('barrier', 1)])
OrderedDict([('rz',

In [116]:
#16,14,11,8
trans_qcs[-1].draw()

»
  ancilla_0 -> 0 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_1 -> 1 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_2 -> 2 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_3 -> 3 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_4 -> 4 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_5 -> 5 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_6 -> 6 ──────────────────────────────────────────────────────────────»
                                                                               »
  ancilla_7 -> 7 ──────────────────────────────────────────────────────────────»
                      ┌────┐      ┌────────────┐      ┌────┐    ┌─────────────┐»
        q_3 -> 8 ─────┤ √X ├──────┤ Rz(3.1416) ├──────┤ √X ├────┤ Rz(-4.8453) ├»
                      └────┘      └────────────┘      └────┘    └─────────────┘»
  ancilla_8 -> 9 ──────────────────────────────────────────────────────────────»
                                                                               »
 ancilla_9 -> 10 ──────────────────────────────────────────────────────────────»
                 ┌─────────────┐      ┌────┐     ┌─────────────┐     ┌────┐    »
       q_2 -> 11 ┤ Rz(-3.1329) ├──────┤ √X ├─────┤ Rz(-1.6224) ├─────┤ √X ├────»
                 └─────────────┘      └────┘     └─────────────┘     └────┘    »
ancilla_10 -> 12 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_11 -> 13 ──────────────────────────────────────────────────────────────»
                      ┌────┐    ┌───────────────┐     ┌────┐                   »
       q_1 -> 14 ─────┤ √X ├────┤ Rz(-0.013636) ├─────┤ √X ├───────────────────»
                      └────┘    └───────────────┘     └────┘                   »
ancilla_12 -> 15 ──────────────────────────────────────────────────────────────»
                  ┌────────────┐      ┌────┐     ┌─────────────┐     ┌────┐    »
       q_0 -> 16 ─┤ Rz(1.5695) ├──────┤ √X ├─────┤ Rz(-1.5708) ├─────┤ √X ├────»
                  └────────────┘      └────┘     └─────────────┘     └────┘    »
ancilla_13 -> 17 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_14 -> 18 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_15 -> 19 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_16 -> 20 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_17 -> 21 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_18 -> 22 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_19 -> 23 ──────────────────────────────────────────────────────────────»
                                                                               »
ancilla_20 -> 24 ──────────────────────────────────────────────────────────────»
                             

In [129]:
job = execute(circuit_list, backend = device, optimization_level = 3, shots = max_shots, seed_transpiler = 0)
print(job.job_id())

6391850df47ae7f5fbf57eb2


In [130]:
job_monitor(job)

Job Status: job has successfully run


In [131]:
# job = device.retrieve_job(job_id='638d16954a111e52c2527afb')

In [132]:
result = job.result()

In [133]:
len(result.get_counts())

125

In [148]:
import json
with open('LiH_auckland_nocut_orign.json', 'w') as f:
    json.dump(result.get_counts(), f)

In [135]:
per_miti_results = []
for i in range(0, 4):
    temp_list = []
    for j in range(0, 25):
        filter_per_miti = norm_dict(filter_results(result.get_counts()[i * 25 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [136]:
len(per_miti_results)

4

In [137]:
for j in range(0, 4):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[j][i], shots = total_counts(per_miti_results[j][i]), Pauli = Pauli_tuple[0])  
    print(final_expect_val)

(-1.0370390814212622+0j)
(-1.037039534719149+0j)
(-0.9351584043522796+0j)
(-1.0091429498937456+0j)


In [138]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(result.get_counts()[i + 4*25], shots = max_shots, Pauli = Pauli_tuple[0]) 

In [139]:
final_expect_val

(-1.0514831765722086+0j)

In [141]:
trans_qc_list = transpile(circuit_list[-25:], device, initial_layout = [16,14,11,8], optimization_level = 3)

In [142]:
mapping = mthree.utils.final_measurement_mapping(trans_qc_list[0])
mapping

{16: 0, 14: 1, 11: 2, 8: 3}

In [144]:
ideal_mapping = {16: 0, 14: 1, 11: 2, 8: 3}
for i in range(0,25):
    mapping = mthree.utils.final_measurement_mapping(trans_qc_list[i])
    if mapping != ideal_mapping:
        print(mapping)

In [145]:
mit = mthree.M3Mitigation(device)

In [146]:
mit.cals_from_system(list(mapping))

In [147]:
mit.cals_to_file(cals_file = 'mit_data')

In [178]:
quasis = mit.apply_correction(result.get_counts()[-25:], [16,14,11,8])
print('Expectation value:',quasis.expval())

Expectation value: [-1.68703627e-02 -1.06504084e-02  6.80432515e-03 -7.17270794e-03 -1.30668921e-02 -3.31188563e-03 -1.92213021e-02  8.96255973e-03 -6.15588134e-03  1.53373339e-02  1.00781741e-02  1.72117845e-05
 -9.69317327e-02 -6.78777728e-03  1.04792981e-02 -9.23811223e-03  4.43871332e-04 -1.95290316e-03 -1.01963757e-02 -2.97378697e-03  4.74601263e-02 -7.44685150e-02 -4.42203799e-03  3.99932353e-03
  9.36352851e-01]


In [163]:
def update_dict(quasis, max_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*max_shots

In [164]:
update_dict(quasis, max_shots = max_shots)

In [165]:
quasis

{'0000': 5.941555923632778,
 '0001': 119.11897850184542,
 '0010': 155.76047398608213,
 '0011': 9676.621368560298,
 '0101': 1.0778291705113112,
 '0110': 1.5884762570138957,
 '0111': -5.335744999084296,
 '1001': -2.4342232193190947,
 '1010': -2.8710660853895122,
 '1011': 47.70884149417368,
 '1101': 0.9831973936591182,
 '1111': 1.8403130165764259}

In [170]:
expect_val = 0
miti_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        quasis = mit.apply_correction(result.get_counts()[i + 4*25], [16,14,11,8])
        update_dict(quasis, max_shots = max_shots)
        print(evaluation(quasis, shots = max_shots, Pauli = Pauli_tuple[0]), evaluation(result.get_counts()[i + 4*25], shots = max_shots, Pauli = Pauli_tuple[0]))
        miti_val += coeff * evaluation(quasis, shots = max_shots, Pauli = Pauli_tuple[0]) 
        expect_val += coeff * evaluation(result.get_counts()[i + 4*25], shots = max_shots, Pauli = Pauli_tuple[0]) 

-0.016870362708540785 -0.0154
-0.01111021882234715 -0.0104
-0.009781220659256639 -0.0094
1.0 1.0
-0.010650408440250772 -0.0098
0.012122660893298986 0.0114
0.002677601387381992 0.0024
-0.06338161635305276 -0.0626
0.06269144374106607 0.0634
-0.9787538304392239 -0.9568
0.006804325152395291 0.0062
0.007073560880938726 0.0066
0.009190951650097713 0.0088
-0.048129562701766666 -0.0486
-0.03617915342830774 -0.0382
0.9946781291577117 0.9692
-0.007172707944366028 -0.0066
-0.013715606293624921 -0.0128
-0.0027119228111350934 -0.0026
-0.012870275791624055 -0.0112
-0.013066892116969967 -0.012
0.006173588371293221 0.0058
-0.005147046413814127 -0.0048
-0.004400777811892402 -0.007
-0.003311885631685314 -0.003
-0.005236745641809069 -0.005
0.019152447772237777 0.0156
0.992210720508353 0.9652
-0.01922130212066907 -0.0172
1.0078692665899223 0.9554
0.00896255972599604 0.0082
0.0023139604676657884 0.0022
0.05218850506334465 0.0486
-0.9808045742291198 -0.9586
-0.00615588133829815 -0.0054
0.9402654005927789 0.

In [175]:
expect_val

(-1.0514831765722086+0j)

In [176]:
miti_val

(-1.076698146893124+0j)

In [177]:
help(mit)

Help on M3Mitigation in module mthree.mitigation object:

class M3Mitigation(builtins.object)
 |  M3Mitigation(system=None, iter_threshold=4096)
 |  
 |  Main M3 calibration class.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, attr)
 |      This allows for checking the status of the threaded cals call
 |      
 |      For certain attr this will join the thread and/or raise an error.
 |  
 |  __init__(self, system=None, iter_threshold=4096)
 |      Main M3 calibration class.
 |      
 |      Parameters:
 |          system (Backend): Target backend.
 |          iter_threshold (int): Sets the bitstring count at which iterative mode
 |                                is turned on (assuming reasonable error rates).
 |      
 |      Attributes:
 |          system (Backend): The target system.
 |          single_qubit_cals (list): 1Q calibration matrices
 |  
 |  apply_correction(self, counts, qubits, distance=None, method='auto', max_iter=25, tol=1e-05, return_mitigation_over